In [10]:
import numpy as np
import re
from numpy import *

In [242]:
import numpy as np
import os
import re
import time
    
from numpy import *
from operator import itemgetter, attrgetter, methodcaller

## regular expression for header parsing ##
re1 = re.compile('ERCC-[0-9]{5}')
re2 = re.compile('[0-9]+')
re3 = re.compile('\*|([0-9]+[MIDNSHPX=])+')
re4 = re.compile('[A-T]{100}')
num_head = 6

def sam_info(root,num_sam,num_fa,num_line):
    for i in range(0,1):
        path = os.path.join(root, 'ERCC%d.fa' %num_fa)
    
    clean_data = open(path,'r')
    clean_seq = clean_data.readlines()
    clean_data.close()
    
    ## make clean dict {'Spike':clean_seq} ##
    spike_idx = [] # to take idx of spike name line #
    clean_dict = {}
    
    for i in range(len(clean_seq)):
        c = re1.search(clean_seq[i])
        if c != None:
            spike_idx += [i]
            if not clean_dict.has_key(c.group()):
                clean_dict.update({c.group():''})
    
    spike_idx += [len(clean_seq)]

    for j in range(len(spike_idx)-1):
        con_clean = ''
        for k in range(spike_idx[j]+1,spike_idx[j+1]):
            con_clean += clean_seq[k][:-1]
        clean_dict[re1.search(clean_seq[spike_idx[j]]).group()]=con_clean
    
    
    for i in range(0,1): # for many files, num_sam can be array(or list)
        path = os.path.join(root, 'ERR%d.sam' %num_sam)
    if num_line != 0:
        noisy_data = open(path,'r')
        noisy_seq = noisy_data.readlines()[94:num_line]
        noisy_data.close()
    elif num_line == 0:
        noisy_data = open(path,'r')
        noisy_seq = noisy_data.readlines()[94:]
        noisy_data.close()
        
    ## make header array ##
    
    err_count=0
    HD1 = []
    HD2 = []
    HD3 = []
    HD4 = []
    HD5 = []
    HD6 = []
    HD7 = []
    
    print "Pre-processing starts..."
    for j in range(len(noisy_seq)):
        if j%100000==0:
            print "Pre-processing is %0.1f%% done.."%(j/float(len(noisy_seq))*100.)
        tmp = noisy_seq[j]
        head1 = re1.search(tmp) # ref. spike name
        head2 = re2.findall(tmp) # match pos. of clean_seq
        head3 = re3.search(tmp) # 30S70M
        head4 = re4.search(tmp) # noisy_seq. length=100
        
        cigar_str = head3.group()
        noisy = head4.group()
        
        if 'D' in cigar_str or 'I' in cigar_str or '*' in cigar_str: # Deletion & Insertion error pop up
            err_count += 1
        elif head1==None: # Error seq pop up
            err_count += 1
        elif 'N' in noisy: # Error seq pop up
            err_count += 1
        else:
            r_name = head1.group()
            tot_seq = clean_dict[r_name]
            tmp_seq = tot_seq[int(head2[4])-1:int(head2[4])-1+100] # for 1-based.
            m_idx = cigar_str.index('M')
            # clear till this
            
            if m_idx == 1: # *M
                match_idx = int(cigar_str[m_idx-1])
                noisy = noisy[:match_idx]
                cut_seq = tmp_seq[:match_idx]
            elif m_idx == 2: # **M
                match_idx = int(cigar_str[m_idx-2]+cigar_str[m_idx-1])
                noisy = noisy[:match_idx]
                cut_seq = tmp_seq[:match_idx]
            elif m_idx == 3: 
                if cigar_str.count('S'): # *S*M
                    s_idx = cigar_str.index('S')
                    match_idx = int(cigar_str[s_idx+1])
                    ss_idx = int(cigar_str[s_idx-1])
                    noisy = noisy[ss_idx:ss_idx+match_idx]
                    cut_seq = tmp_seq[:match_idx]
                else:
                    match_idx = 100 # ***M
                    noisy = noisy[:match_idx]
                    cut_seq = tmp_seq[:match_idx]
            elif m_idx == 4: 
                if cigar_str.count('S'): 
                    s_idx = cigar_str.index('S')
                    if s_idx == 1: # *S**M*S
                        match_idx = int(cigar_str[2]+cigar_str[3])
                        ss_idx = int(cigar_str[0])
                        noisy = noisy[ss_idx:ss_idx+match_idx]
                        cut_seq = tmp_seq[:match_idx]
                    elif s_idx == 2: # **S*M
                        match_idx = int(cigar_str[s_idx+1])
                        ss_idx = int(cigar_str[s_idx-2]+cigar_str[s_idx-1])
                        noisy = noisy[ss_idx:ss_idx+match_idx]
                        cut_seq = tmp_seq[:match_idx]
            elif m_idx == 5: # **S**M
                if cigar_str.count('S'):
                    s_idx = cigar_str.index('S')
                    match_idx = int(cigar_str[s_idx+1]+cigar_str[s_idx+2])
                    ss_idx = int(cigar_str[s_idx-2]+cigar_str[s_idx-1])
                    noisy = noisy[ss_idx:ss_idx+match_idx]
                    cut_seq = tmp_seq[:match_idx]
            else:
                print "There is another case!!"
                break
            print r_name, head2[4], cigar_str
            print cut_seq
            print noisy, len(cut_seq)
            HD1.append(r_name)
            HD2.append(head2[4])
            HD3.append(cigar_str)
            HD4.append(cut_seq)
            HD5.append(noisy)
            HD6.append(len(cut_seq))
            
    reshaped1 = np.reshape(HD1, (len(noisy_seq)-err_count, 1))
    reshaped2 = np.reshape(HD2, (len(noisy_seq)-err_count, 1))
    reshaped3 = np.reshape(HD3, (len(noisy_seq)-err_count, 1))
    reshaped4 = np.reshape(HD4, (len(noisy_seq)-err_count, 1))
    reshaped5 = np.reshape(HD5, (len(noisy_seq)-err_count, 1))
    reshaped6 = np.reshape(HD6, (len(noisy_seq)-err_count, 1))
    
    new_arr = np.hstack((reshaped1,reshaped2,reshaped3,reshaped4,reshaped5,reshaped6))
    new_arr = sorted(new_arr, key=itemgetter(0))
    new_arr = np.reshape(new_arr, (len(noisy_seq)-err_count, num_head))
    print "Pre-processing is completed!!"
    return new_arr # list of strs

def make_PI(new_arr):
    PI = zeros((4,4))
    dict_A = {'A':0, 'T':0, 'G':0, 'C':0}
    dict_T = {'A':0, 'T':0, 'G':0, 'C':0}
    dict_G = {'A':0, 'T':0, 'G':0, 'C':0}
    dict_C = {'A':0, 'T':0, 'G':0, 'C':0}
    tot_dict = [dict_A, dict_T, dict_G, dict_C]
    print "Making PI starts..."
    for i in range(len(new_arr)):
        if i%100000==0:
            print "Making PI is %0.1f%% done.."%(i/float(len(new_arr))*100.) 
        for j in range(len(new_arr[i][3])):
            if new_arr[i][3][j] == 'A':
                if new_arr[i][4][j] == 'A':
                    dict_A[new_arr[i][4][j]] += 1
                elif new_arr[i][4][j] == 'T':
                    dict_A[new_arr[i][4][j]] += 1
                elif new_arr[i][4][j] == 'G':
                    dict_A[new_arr[i][4][j]] += 1
                elif new_arr[i][4][j] == 'C':
                    dict_A[new_arr[i][4][j]] += 1
                else:
                    print "Error 1"
                    break
            
            elif new_arr[i][3][j] == 'T':
                if new_arr[i][4][j] == 'A':
                    dict_T[new_arr[i][4][j]] += 1
                elif new_arr[i][4][j] == 'T':
                    dict_T[new_arr[i][4][j]] += 1
                elif new_arr[i][4][j] == 'G':
                    dict_T[new_arr[i][4][j]] += 1
                elif new_arr[i][4][j] == 'C':
                    dict_T[new_arr[i][4][j]] += 1
                else:
                    print "Error 2"
                    break
            
            elif new_arr[i][3][j] == 'G':
                if new_arr[i][4][j] == 'A':
                    dict_G[new_arr[i][4][j]] += 1
                elif new_arr[i][4][j] == 'T':
                    dict_G[new_arr[i][4][j]] += 1
                elif new_arr[i][4][j] == 'G':
                    dict_G[new_arr[i][4][j]] += 1
                elif new_arr[i][4][j] == 'C':
                    dict_G[new_arr[i][4][j]] += 1
                else:
                    print "Error 3"
                    break
            
            elif new_arr[i][3][j] == 'C':
                if new_arr[i][4][j] == 'A':
                    dict_C[new_arr[i][4][j]] += 1
                elif new_arr[i][4][j] == 'T':
                    dict_C[new_arr[i][4][j]] += 1
                elif new_arr[i][4][j] == 'G':
                    dict_C[new_arr[i][4][j]] += 1
                elif new_arr[i][4][j] == 'C':
                    dict_C[new_arr[i][4][j]] += 1
                else:
                    print "Error 4"
                    break
            else:
                print "Error 5"
                break
    
    for p in range(PI.shape[0]):
        PI[p,0] = tot_dict[p]['A']
        PI[p,1] = tot_dict[p]['T']
        PI[p,2] = tot_dict[p]['G']
        PI[p,3] = tot_dict[p]['C']
    for q in range(PI.shape[0]):
        PI[q] = PI[q]/float(sum(PI[q]))
    print "Making PI is completed!!"
    return PI

In [6]:
ll = 'GCTGTTCGATTTCTTAAAGGACTGAATATTCGGTGGCA'
pp = 'GCTGTTGTAGGTATCGGAGAAATGAGCGGTGAATGACA'
mm = 'GCTGTTCGATTTCTTAAAGGACTGAATATTCGGTGGCA'
tmp_1 = 0
tmp_2 = 0
for i in range(len(ll)):
    if ll[i]!=mm[i]:
        tmp_1 += 1
for j in range(len(pp)):
    if pp[j]!=mm[j]:
        tmp_2 +=1
print tmp_1
print tmp_2

0
18


In [121]:
p = []
for k in clean_.keys():
    p += [k, len(clean_[k])]
p = np.reshape(p, (len(clean_),2))
q = sorted(p, key=itemgetter(0))
print q

[array(['ERCC-00002', '1061'], 
      dtype='|S10'), array(['ERCC-00003', '1023'], 
      dtype='|S10'), array(['ERCC-00004', '523'], 
      dtype='|S10'), array(['ERCC-00009', '984'], 
      dtype='|S10'), array(['ERCC-00012', '994'], 
      dtype='|S10'), array(['ERCC-00013', '808'], 
      dtype='|S10'), array(['ERCC-00014', '1957'], 
      dtype='|S10'), array(['ERCC-00016', '844'], 
      dtype='|S10'), array(['ERCC-00017', '1136'], 
      dtype='|S10'), array(['ERCC-00019', '644'], 
      dtype='|S10'), array(['ERCC-00022', '751'], 
      dtype='|S10'), array(['ERCC-00024', '536'], 
      dtype='|S10'), array(['ERCC-00025', '1994'], 
      dtype='|S10'), array(['ERCC-00028', '1130'], 
      dtype='|S10'), array(['ERCC-00031', '1138'], 
      dtype='|S10'), array(['ERCC-00033', '2022'], 
      dtype='|S10'), array(['ERCC-00034', '1019'], 
      dtype='|S10'), array(['ERCC-00035', '1130'], 
      dtype='|S10'), array(['ERCC-00039', '740'], 
      dtype='|S10'), array(['ERCC-00040',

In [89]:
clean_ = noisy_head_info('./',1211177,92, 200000)

In [146]:
print clean_['ERCC-00130']
print clean_['ERCC-00130'][990:1090]
print len(clean_['ERCC-00130'][990:1090])
print len('GATTCAGCGCTATTTGCGCCAAGAACTGTTAACGTC')

GCATTTTGAAAATTCTATGGAAGAGCTAGCATCTCTGACGAAAACAGCAGACGGAAAAGTACTGACCAGCGTCACACAAAAACGGAACAGGGCTGACGCCGCTACATATATAGGAAAAGGGAAGGTAGAAGAGCTGAAGGCACTCGTGGAAGAGCTTGAAGCTGATCTCCTCATCTTTAATGATGAACTGTCGCCAAGTCAGCTGAAGTCATTGGCAACAGCAATTGAAGTGAAGATGATTGACCGCACGCAATTGATATTAGATATTTTTGCAAAGCGGGCGAGAACGAGAGAAGGCAAACTTCAAATTGAGCTGGCTCAGCTGCAATATGCACTGCCGCGTCTGACGGGACAAGGGATCAACCTTTCCCGGCAAGGCGGAGGAATTGGGGCAAGAGGTCCCGGGGAAACGAAACTGGAAACCGACCGCCGCCATATCAGAAATCGCATTCATGAAATCAACACACAGCTTTCCACTGTCATTCGCCATAGAAGCCGATACCGTGAAAGAAGAAAGAAAAACGGTGTGCTTCAAATTGCGCTTGTCGGCTATACAAACGCAGGGAAATCAACATGGTTCAACCGCCTGACGAGTGCTGACAGCTATGAAGAAGACCTCCTGTTTGCCACGCTGGACCCGATGACCAGAAAAATGGTCCTGCCAAGCGGCTACAGTGTTCTTCTTTCAGATACAGTAGGATTTATTCAGGATCTTCCGACGACATTGATTGCTGCATTCCGCTCAACGCTTGAGGAAGTAAAAGAAGCGGATTTAATTCTGCATTTAATTGATTCTTCAAATGAGGATTATGCGGGACATGAAAAAACAGTGCTTCGGCTGCTTGAGGAGCTTGAAGCAGATGATATCCCGATGCTGACGGCTTACAATAAACGTGATCAAAAACTGCCTGATTTTATACCGACCGCCGGAAGGGATCACATTATGGTCAGTGCGAAATTTGAGGACGACGCTGCAGCCTTTAAAGAAGCGATTCAGCGC

In [244]:
print b

[[  9.96837123e-01   6.55056533e-04   1.56205789e-03   9.45762094e-04]
 [  5.77221400e-04   9.97069800e-01   9.09925402e-04   1.44305350e-03]
 [  1.00702169e-03   7.87307865e-04   9.97693005e-01   5.12665586e-04]
 [  6.51878596e-04   1.17611663e-03   4.14831834e-04   9.97757173e-01]]


In [192]:
print clean_['ERCC-00004'], len(clean_['ERCC-00004'])
print clean_['ERCC-00004'][114:214], len(clean_['ERCC-00004'][114:214])
print clean_['ERCC-00004'][114:214][65:]

TCTTGCTTCAACAATAACGTCTCTTTCAGAAGGCATTGGTATCTTTTCCCCACTTCCAAGCATTTTTTCAACTAATCTTATGTTATTAACCATTTCCTTAAATTCTTCTGGGTCTGCTGACAAAGCATGATCAGGACCTTCCATATTTTTATCTAAGGTAAAGTGCTTCTCAATAACATCCGCTCCTAAGGCAACAGAAACTACTGGGGCGAGTATTCCCAATGTATGGTCAGAATATCCCACAGGGATATTGAATATACTTTTCAAGGTTTTAATAGCGTTTAAATTGACATCTTCATAAGGGGTTGGGTAAGATGAAATACAATGCAATAAAATAATATCCCTGCATCCATTATTTTCTAAAACTTTAACTGCTTCCCAAATTTCCCCAATATCAGACATTCCTGTAGATAAAATCACCGGCTTGCCTGTTTTTGCCACTTTTTCTAATAAGGGATAAAAGGTTAAATCACCAGAGGCAATTTTAAATCAGGCACATAAAAAAAAAAAAAAAAAAAAAAAA 523
TGCTGACAAAGCATGATCAGGACCTTCCATATTTTTATCTAAGGTAAAGTGCTTCTCAATAACATCCGCTCCTAAGGCAACAGAAACTACTGGGGCGAGT 100
CCGCTCCTAAGGCAACAGAAACTACTGGGGCGAGT


In [194]:
o = '65S35M'
if o.count('M'):
    print '1'
else:
    print '0'

1


In [ ]:
CACCAGAGGCAATTTTAAATCAGGCACATAAAAAA
CACCAGAGGCAATTTTAAATCAGGCACTTAAAAAA

In [102]:
print clean_['ERCC-00004'], len(clean_['ERCC-00004'])
print clean_['ERCC-00004'][470:], len(clean_['ERCC-00004'][470:])
print clean_['ERCC-00004'][470:570]
print clean_['ERCC-00004'][470:570][65:]

TCTTGCTTCAACAATAACGTCTCTTTCAGAAGGCATTGGTATCTTTTCCCCACTTCCAAGCATTTTTTCAACTAATCTTATGTTATTAACCATTTCCTTAAATTCTTCTGGGTCTGCTGACAAAGCATGATCAGGACCTTCCATATTTTTATCTAAGGTAAAGTGCTTCTCAATAACATCCGCTCCTAAGGCAACAGAAACTACTGGGGCGAGTATTCCCAATGTATGGTCAGAATATCCCACAGGGATATTGAATATACTTTTCAAGGTTTTAATAGCGTTTAAATTGACATCTTCATAAGGGGTTGGGTAAGATGAAATACAATGCAATAAAATAATATCCCTGCATCCATTATTTTCTAAAACTTTAACTGCTTCCCAAATTTCCCCAATATCAGACATTCCTGTAGATAAAATCACCGGCTTGCCTGTTTTTGCCACTTTTTCTAATAAGGGATAAAAGGTTAAATCACCAGAGGCAATTTTAAATCAGGCACATAAAAAAAAAAAAAAAAAAAAAAAA 523
CACCAGAGGCAATTTTAAATCAGGCACATAAAAAAAAAAAAAAAAAAAAAAAA 53
CACCAGAGGCAATTTTAAATCAGGCACATAAAAAAAAAAAAAAAAAAAAAAAA



In [240]:
print b

[[  9.96346610e-01   6.92334068e-04   1.74517959e-03   1.21587642e-03]
 [  6.01606684e-04   9.96712083e-01   1.12775923e-03   1.55855110e-03]
 [  1.12670135e-03   9.88382875e-04   9.97207196e-01   6.77719854e-04]
 [  8.89458310e-04   1.08873463e-03   6.35173656e-04   9.97386633e-01]]


In [69]:
c='CACCAGAGGCAATTTTAAATCAGGCACTTAAAAAA'
print len(c)

35


In [18]:
print a[46881]
print a[852]

['ERCC-00035' '790' '2S98M'
 'GATGGGCGACACATGTCTACCCTGGCATAACGCCCAGCTTGATAAGAGTGAAACGACCCATCTGACACAGGCCGGCATAGAAGACCAGCGCAATGCGA'
 'AAGATGGGCGACACATGTCTACCCTCGAATAACGCCCAGCTTGATAAGAGTGAAACGACCCATCGGACACAGGCCGGCATAGAAGACCAGCGCAATGC'
 '98' '98']
['ERCC-00002' '758' '100M'
 'GTGTACACCCTTTTGAGCAATGATTGCACAACCTGCGATCACCTTATACAGAATTATCAATCAAGCTCCCCGAGGAGCGGACTTGTAAGGACCGCCGCTT'
 'GTGTACACCCTTTTGAGCAATGATTGCACAATCTGCGATCACCTTATACAGAATTATCAATCAAGCTCCCCGAGGAGCGGACTTGTAAGGACCGCCGCTT'
 '100' '100']


In [57]:
def clean(root, num_fa):
    for i in range(0,1):
        path = os.path.join(root, 'ERCC%d.fa' %num_fa)
    
    clean_data = open(path,'r')
    clean_seq = clean_data.readlines()
    clean_data.close()
    
    ## make clean dict {'Spike':clean_seq} ##
    spike_idx = [] # to take idx of spike name line #
    clean_dict = {}
    
    for i in range(len(clean_seq)):
        c = re1.search(clean_seq[i])
        if c != None:
            spike_idx += [i]
            if not clean_dict.has_key(c.group()):
                clean_dict.update({c.group():''})
    
    spike_idx += [len(clean_seq)]

    for j in range(len(spike_idx)-1):
        con_clean = ''
        for k in range(spike_idx[j]+1,spike_idx[j+1]):
            con_clean += clean_seq[k][:-1]
        clean_dict[re1.search(clean_seq[spike_idx[j]]).group()]=con_clean
    return clean_dict

In [58]:
c = clean('./',92)

In [66]:
c['ERCC-00130']

'TCCAGATTACTTCCATTTCCGCCCAAGCTGCTCACAGTATACGGGCGTCGGCATCCAGACCGTCGGCTGATCGTGGTTTTACTAGGCTAGACTAGCGTACGAGCACTATGGTCAGTAATTCCTGGAGGAATAGGTACCAAGAAAAAAACGAACCTTTGGGTTCCAGAGCTGTACGGTCGCACTGAACTCGGATAGGTCTCAGAAAAACGAAATATAGGCTTACGGTAGGTCCGAATGGCACAAAGCTTGTTCCGTTAGCTGGCATAAGATTCCATGCCTAGATGTGATACACGTTTCTGGAAACTGCCTCGTCATGCGACTGTTCCCCGGGGTCAGGGCCGCTGGTATTTGCTGTAAAGAGGGGCGTTGAGTCCGTCCGACTTCACTGCCCCCTTTCAGCCTTTTGGGTCCTGTATCCCAATTCTCAGAGGTCCCGCCGTACGCTGAGGACCACCTGAAACGGGCATCGTCGCTCTTCGTTGTTCGTCGACTTCTAGTGTGGAGACGAATTGCCAGAATTATTAACTGCGCAGTTAGGGCAGCGTCTGAGGAAGTTTGCTGCGGTTTCGCCTTGACCGCGGGAAGGAGACATAACGATAGCGACTCTGTCTCAGGGGATCTGCATATGTTTGCAGCATACTTTAGGTGGGCCTTGGCTTCCTTCCGCAGTCAAAACCGCGCAATTATCCCCGTCCTGATTTACTGGACTCGCAACGTGGGTCCATCAGTTGTCCGTATACCAAGACGTCTAAGGGCGGTGTACACCCTTTTGAGCAATGATTGCACAACCTGCGATCACCTTATACAGAATTATCAATCAAGCTCCCCGAGGAGCGGACTTGTAAGGACCGCCGCTTTCGCTCGGGTCTGCGGGTTATAGCTTTTCAGTCTCGACGGGCTAGCACACATCTGGTTGACTAGGCGCATAGTCGCCATTCACAGATTTGCTCGGCAATCAGTACTGGTAGGCGTTAGACCCCGTGACTCGTGGCTGAACGG

In [68]:
tmp[1]+tmp[2]+tmp[3]

'TCCAGATTACTTCCATTTCCGCCCAAGCTGCTCACAGTATACGGGCGTCG\nGCATCCAGACCGTCGGCTGATCGTGGTTTTACTAGGCTAGACTAGCGTAC\nGAGCACTATGGTCAGTAATTCCTGGAGGAATAGGTACCAAGAAAAAAACG\n'

In [29]:
with open('./ERCC92.fa', 'r') as test:
    tmp = test.readlines()

In [123]:
tmp[1]


'TCCAGATTACTTCCATTTCCGCCCAAGCTGCTCACAGTATACGGGCGTCG\n'

In [71]:
with open('./ERR1211177.sam', 'r') as tt:
    yeah = tt.readlines()

In [82]:
print a[62]

['ERCC-00002' '129' '100M'
 'AATAGGTACCAAGAAAAAAACGAACCTTTGGGTTCCAGAGCTGTACGGTCGCACTGAACTCGGATAGGTCTCAGAAAAACGAAATATAGGCTTACGGTAG'
 'AATAGGTACCAAGAAAAAAACGAACCTTTGGGTTCCAGAGCTGTACGGTCGCACTGAACTCGGATAGGTCTCAGAAAAACGAAATATAGGCTTACGGTAG'
 '100' '100']


In [159]:
for i in range(94,150):
    print yeah[i], re2.findall(yeah[i])[4]

ERR1211177.24163	67	ERCC-00046	93	40	38M62S	=	93	200	GCTGTTCGATTTCTTAAAGGACTGAATATTCGGTGGCATGGGGAGAAATGACACACTTACTACTTGGATGTCAGCTGAAGACTCCTGACGTAATCTCTTC	@BCFDDFFHGHHHIJIJIIIGIGJGGIIIJJJIFGHIEHIJIIGHGEGHIFGGHGIIIJGIJJIJHHHFHFFFFFFFDEECCCCCDCCCD<<?BDDDA@@	MD:Z:38	NH:i:1	HI:i:1	NM:i:0	SM:i:40	XQ:i:40	X2:i:0	XO:Z:PI	PG:Z:T
93
ERR1211177.24163	131	ERCC-00046	93	40	38M62S	=	93	-200	GCTGTTCGATTTCTTAAAGGACTGAATATTCGGTGGCATGGGGAGAAATGACACACTTACTACTTGGATGTCAGCTGAAGACTCCTGACGTAATCTCTTC	@BCFDDFFHGHHHIJIJIIIGIGJGGIIIJJJIFGHIEHIJIIGHGEGHIFGGHGIIIJGIJJIJHHHFHFFFFFFFDEECCCCCDCCCD<<?BDDDA@@	MD:Z:38	NH:i:1	HI:i:1	NM:i:0	SM:i:40	XQ:i:40	X2:i:0	XO:Z:PI	PG:Z:T
93
ERR1211177.38733	67	ERCC-00130	675	40	100M	=	675	200	GTGTTCTTCTTTCAGATACAGTAGGATTTATTCAGGATCTTCCGACGACATTGATTGCTGCATTCCGCTCAACGCTTGAGGAAGTAAAAGAAGCGGATTT	B?@DDDBDFHDFHHIEBGHCHCCFGF>CEHBHIEEGHGIJIIGHGIIGHGAHIIICDHCHCGGCEHFGEFFFCCCDDBDDDABD>(5@@3<8?3><99<>	MD:Z:100	NH:i:1	HI:i:1	NM:i:0	SM:i:40	XQ:i:40	X2:i:0	XO:Z:PI	PG:Z:A
675
ERR1211177.38733	131	

In [127]:
for i in range(300):
    print yeah[i][17:180]

orted

snap	VN:2014-08-20	CL:/mnt/d/bin/gsnap-2014-08-20/bin/gsnap --gunzip -A sam -B 5 -n 1 -Q --nofails -D /mnt/d/bin/gsnap-2014-08-20/share/ERCC92 -d ERCC92 ERR1211177
	LN:1061

	LN:1023

	LN:523

	LN:984

	LN:994

	LN:808

	LN:1957

	LN:844

	LN:1136

	LN:644

	LN:751

	LN:536

	LN:1994

	LN:1130

	LN:1138

	LN:2022

	LN:1019

	LN:1130

	LN:740

	LN:744

	LN:1122

	LN:1023

	LN:1023

	LN:1156

	LN:522

	LN:992

	LN:274

	LN:1023

	LN:274

	LN:1021

	LN:1136

	LN:525

	LN:523

	LN:1136

	LN:1023

	LN:644

	LN:1137

	LN:642

	LN:603

	LN:522

	LN:1023

	LN:642

	LN:273

	LN:993

	LN:644

	LN:534

	LN:1022

	LN:994

	LN:844

	LN:1020

	LN:1124

	LN:521

	LN:1107

	LN:523

	LN:1143

	LN:1350

	LN:2022

	LN:1022

	LN:536

	LN:994

	LN:1136

	LN:840

	LN:1991

	LN:1136

	LN:536

	LN:1022

	LN:1118

	LN:1059

	LN:771

	LN:274

	LN:1033

	LN:537

	LN:1024

	LN:493

	LN:784

	LN:538

	LN:1042

	LN:1023

	LN:494

	LN:743

	LN:537

	LN:494

	LN:1019

	LN:1027

	LN:743

	LN:523

	LN:543

	LN:1

In [128]:
for i in range(94,300):
    print re2.findall(yeah[i][17:180])[2]

93
93
675
675
414
414
630
630
872
872
877
877
162
162
505
505
321
321
887
887
372
372
290
290
268
268
273
273
276
276
286
286
286
286
276
276
261
261
293
293
292
292
319
319
318
318
273
273
193
193
1000
1000
1856
1856
351
351
822
822
828
828
694
694
375
375
405
405
115
115
360
360
339
339
807
807
267
267
386
386
148
148
577
577
531
531
0
0
365
365
994
994
912
912
425
425
683
683
109
109
818
818
314
314
321
321
273
273
68
68
307
307
667
667
572
572
692
692
17
17
201
201
330
330
80
80
228
228
740
740
928
928
45
45
693
693
427
427
28
28
685
685
494
494
222
222
13
13
529
529
748
748
888
888
115
115
222
222
188
188
531
531
837
837
369
369
157
157
972
972
130
130
883
883
281
281
222
222
942
942
85
85
314
314
286
286
489
489
851
851
306
306
448
448
586
586
380
380
823
823
612
612
544
544
790
790
670
670


In [87]:
for i in range(50,250):
    print a[i]
#1S97M2S, 2S98M, 


['ERCC-00002' '697' '100M'
 'GATTTACTGGACTCGCAACGTGGGTCCATCAGTTGTCCGTATACCAAGACGTCTAAGGGCGGTGTACACCCTTTTGAGCAATGATTGCACAACCTGCGAT'
 'GATTTACTGGACTCGCAACGTGGGTCCATCAGTTGTCCGTATACCAAGACGTCTAAGGGCGGTGTACACCCTTTTGAGCAATGATTGCACAACCTGCGAT'
 '100' '100']
['ERCC-00002' '697' '100M'
 'GATTTACTGGACTCGCAACGTGGGTCCATCAGTTGTCCGTATACCAAGACGTCTAAGGGCGGTGTACACCCTTTTGAGCAATGATTGCACAACCTGCGAT'
 'GATTTACTGGACTCGCAACGTGGGTCCATCAGTTGTCCGTATACCAAGACGTCTAAGGGCGGTGTACACCCTTTTGAGCAATGATTGCACAACCTGCGAT'
 '100' '100']
['ERCC-00002' '819' '9S91M'
 'CGAGGAGCGGACTTGTAAGGACCGCCGCTTTCGCTCGGGTCTGCGGGTTATAGCTTTTCAGTCTCGACGGGCTAGCACACATCTGGTTGAC'
 'CAAGCTCCCCGAGGAGCGGACTTGTAAGGACCGCCGCTTTCGCTCGGGTCTGCGGGTTATAGCTTTTCAGTCTCGACGGGCTAGCACACAT'
 '91' '91']
['ERCC-00002' '819' '9S91M'
 'CGAGGAGCGGACTTGTAAGGACCGCCGCTTTCGCTCGGGTCTGCGGGTTATAGCTTTTCAGTCTCGACGGGCTAGCACACATCTGGTTGAC'
 'CAAGCTCCCCGAGGAGCGGACTTGTAAGGACCGCCGCTTTCGCTCGGGTCTGCGGGTTATAGCTTTTCAGTCTCGACGGGCTAGCACACAT'
 '91' '91']
['ERCC-00002' '669' '100M'
 'GTCAAAACCGCGC

In [46]:
k = 'GGCTAGACTAGCGTACGAGCACT'
print len(k)

23


In [ ]:
GATGGGCGACACATGTCTACCCTGGCATAACGCCCAGCTTGATAAGAGTGAAACGACCCATCTGACACAGGCCGGCATAGAAGACCAGCGCAATGCGA
GATGGGCGACACATGTCTACCCTCGAATAACGCCCAGCTTGATAAGAGTGAAACGACCCATCGGACACAGGCCGGCATAGAAGACCAGCGCAATGC

In [32]:
nb_classes = 4
nt_order = "ATGC"
PI = array([[0.9938, 0.0005, 0.0049, 0.0008],
          [0.0030, 0.9880, 0.0034, 0.0056],
          [0.0084, 0.0053, 0.9816, 0.0046],
          [0.0006, 0.0042, 0.0008, 0.9944]])
H = linalg.inv(PI)
LAMBDA = array([[0, 1, 1, 1],
              [1, 0, 1, 1],
              [1, 1, 0, 1],
              [1, 1, 1, 0]])
S = zeros((nb_classes, nb_classes**nb_classes),dtype=int)
for s in range(1,nb_classes**nb_classes):
    for x in range(4):
        S[x][s] = S[x][s-1]
    
    if S[0][s] != 3:
        S[0][s] += 1
        continue
    else:
        S[0][s] = 0#
        
    if S[1][s] != 3:
        S[1][s] += 1
        continue
    else:
        S[1][s] = 0
        
    if S[2][s] != 3:
        S[2][s] += 1
        continue
    else:
        S[2][s] = 0
        
    if S[3][s] != 3:
        S[3][s] += 1
        continue
    else:
        print "ERROR: S index out of bound!\n"

RHO   = zeros((nb_classes, nb_classes**nb_classes),dtype=float)

RHO_R = zeros((nb_classes, nb_classes*nb_classes),dtype=float)

for s in range(nb_classes**nb_classes):
    for x in range(4):
        RHO[x][s] = (PI[x][0]*LAMBDA[x][S[0][s]] + PI[x][1]*LAMBDA[x][S[1][s]] +
                     PI[x][2]*LAMBDA[x][S[2][s]] + PI[x][3]*LAMBDA[x][S[3][s]])
for s in range(nb_classes*nb_classes):
    for x in range(4):
        RHO_R[x][s] = PI[x][int(s/4)]*LAMBDA[x][s%4]
L=dot(H,RHO) 
L_new=-L+amax(L)     # A new loss matrix
L_R = dot(H,RHO_R)
L_R_new=-L_R+amax(L_R)
k_max=26
print L, L.shape
print L_new, L_new.shape
print L_R_new, L_R_new.shape


[[-0.00628304  0.99376248  0.99380322 ...,  1.0007846   1.00082534
   1.00078364]
 [ 1.00302262  0.99698217  1.00004802 ...,  1.00264165  1.0057075
   1.00568163]
 [ 1.00859215  1.00006959  0.99149526 ...,  1.00468888  0.99611455
   1.00467537]
 [ 1.00058749  1.00001645  1.00001038 ..., -0.00504356 -0.00504964
  -0.00565977]] (4, 256)
[[ 1.02488897  0.02484345  0.02480271 ...,  0.01782133  0.01778059
   0.01782229]
 [ 0.01558331  0.02162376  0.0185579  ...,  0.01596428  0.01289843
   0.0129243 ]
 [ 0.01001378  0.01853633  0.02711066 ...,  0.01391704  0.02249137
   0.01393056]
 [ 0.01801844  0.01858947  0.01859555 ...,  1.02364949  1.02365556
   1.0242657 ]] (4, 256)
[[  1.00008626e+00   4.07387350e-05   0.00000000e+00   4.17055150e-05
    1.00054532e+00   9.99568936e-01   1.00001556e+00   1.00003888e+00
    1.00497296e+00   1.00004055e+00   9.95113644e-01   1.00004155e+00
    1.00084720e+00   1.00003949e+00   1.00001908e+00   9.99262928e-01]
 [  9.97038299e-01   1.00307875e+00   1.0000

In [41]:
a = zeros(5)
print a

[ 0.  0.  0.  0.  0.]


In [26]:
def make_data_for_ndude_new3(lines,k,L,L_R,nt_order):
    seq = lines
    tot_syms = len(seq)
    C = zeros((tot_syms-2*k, 2*k*4),dtype=int)
    Z = zeros((tot_syms, 4),dtype=int)
    Y = zeros((tot_syms-2*k, 4**4),dtype=int)
    Y_R = zeros((tot_syms-2*k, 4*4),dtype=int)
    for i in range(tot_syms):
        if i<=(tot_syms-2):
            for j in range(4):
                if seq[i] == nt_order[j]:
                    seq = seq[:i] + str(j) + seq[i+1:]
        else:
            for j in range(4):
                if seq[i] == nt_order[j]:
                    seq = seq[:i] + str(j)
        Z[i,int(seq[i])] = 1
    
    for i in range(k,tot_syms-k):
        c_i = vstack((Z[i-k:i],Z[i+1:i+k+1])).reshape(1,2*k*4)
        C[i-k,] = c_i
        Y[i-k,] = dot(Z[i],L)
        Y_R[i-k,] = dot(Z[i],L_R)
       
    print C, C.shape,"\n"
    print Y, Y.shape,"\n"
    print Y_R, Y_R.shape
    return lines, C, Y, Y_R



In [40]:
lines = 'ATGCAATTGGCAGCT\n'
print lines[-1]

In [36]:
lines = 'ATGCGGTATGATCCGGAAATATACCGCGCTGTGATC'
print len(lines)
tmp = [1, 0, 0, 0]
tmp2 = dot(tmp,L_new)
print tmp2, tmp2.shape
tmp3 = dot(tmp,L_R_new)
print tmp3, tmp3.shape

 36
[ 1.02488897  0.02484345  0.02480271  0.02484441  1.02391258  0.02386707
  0.02382633  0.02386803  1.02435921  0.02431369  0.02427295  0.02431466
  1.02438253  0.02433701  0.02429627  0.02433798  1.01995655  0.01991103
  0.01987029  0.019912    1.01898017  0.01893465  0.01889391  0.01893562
  1.0194268   0.01938128  0.01934054  0.01938225  1.01945012  0.0194046
  0.01936386  0.01940556  1.01502965  0.01498413  0.01494339  0.01498509
  1.01405327  0.01400775  0.01396701  0.01400871  1.0144999   0.01445438
  0.01441364  0.01445534  1.01452321  0.01447769  0.01443696  0.01447866
  1.01995755  0.01991203  0.01987129  0.019913    1.01898117  0.01893565
  0.01889491  0.01893662  1.0194278   0.01938228  0.01934154  0.01938325
  1.01945112  0.0194056   0.01936486  0.01940657  1.02408126  0.02403574
  0.023995    0.0240367   1.02310488  0.02305936  0.02301862  0.02306032
  1.0235515   0.02350598  0.02346525  0.02350695  1.02357482  0.0235293
  0.02348857  0.02353027  1.01914884  0.01910332 

In [28]:
L, context, label1, label2 = make_data_for_ndude_new3(lines, 1, L,L_R,nt_order)

[[1 0 0 0 0 0 1 0]
 [0 1 0 0 0 0 0 1]
 [0 0 1 0 0 0 1 0]
 [0 0 0 1 0 0 1 0]
 [0 0 1 0 0 1 0 0]
 [0 0 1 0 1 0 0 0]
 [0 1 0 0 0 1 0 0]
 [1 0 0 0 0 0 1 0]
 [0 1 0 0 1 0 0 0]
 [0 0 1 0 0 1 0 0]
 [1 0 0 0 0 0 0 1]
 [0 1 0 0 0 0 0 1]
 [0 0 0 1 0 0 1 0]
 [0 0 0 1 0 0 1 0]
 [0 0 1 0 1 0 0 0]
 [0 0 1 0 1 0 0 0]
 [1 0 0 0 1 0 0 0]
 [1 0 0 0 0 1 0 0]
 [1 0 0 0 1 0 0 0]
 [0 1 0 0 0 1 0 0]
 [1 0 0 0 1 0 0 0]
 [0 1 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 1]
 [0 0 0 1 0 0 1 0]
 [0 0 0 1 0 0 0 1]
 [0 0 1 0 0 0 1 0]
 [0 0 0 1 0 0 0 1]
 [0 0 1 0 0 1 0 0]
 [0 0 0 1 0 0 1 0]
 [0 1 0 0 0 1 0 0]
 [0 0 1 0 0 0 1 0]
 [0 1 0 0 1 0 0 0]
 [0 0 1 0 0 1 0 0]
 [1 0 0 0 0 0 0 1]] (34, 8) 

[[1 0 1 ..., 1 1 1]
 [1 1 0 ..., 1 0 1]
 [1 1 1 ..., 0 0 0]
 ..., 
 [1 1 0 ..., 1 0 1]
 [0 0 0 ..., 1 1 1]
 [1 0 1 ..., 1 1 1]] (34, 256) 

[[0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0]
 [0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0]
 [0 0 0 0 1 0 1 1 0 0 0 0 0 

In [18]:
a = 0
for i in range(5):
    a += i
    print a
    if a==3:
        print "loopbreak"
        break
    else:
        print "check2"

0
check2
1
check2
3
loopbreak


In [19]:
import numpy as np
import os
import re
import time
import timeit
    
from numpy import *
from operator import itemgetter, attrgetter, methodcaller

## regular expression for header parsing ##
re1 = re.compile('ERCC-[0-9]{5}')
re2 = re.compile('[0-9]+')
re3 = re.compile('\*|([0-9]+[MIDNSHPX=])+')
re4 = re.compile('[A-T]{100}')
num_head = 6

def noisy_head_info(root,num_sam,num_fa):
    for i in range(0,1):
        path = os.path.join(root, 'ERCC%d.fa' %num_fa)
    
    clean_data = open(path,'r')
    clean_seq = clean_data.readlines()
    clean_data.close()
    
    ## make clean dict {'Spike':clean_seq} ##
    spike_idx = [] # to take idx of spike name line #
    clean_dict = {}
    
    for i in range(len(clean_seq)):
        c = re1.search(clean_seq[i])
        if c != None:
            spike_idx += [i]
            if not clean_dict.has_key(c.group()):
                clean_dict.update({c.group():''})
    
    spike_idx += [len(clean_seq)]

    for j in range(len(spike_idx)-1):
        con_clean = ''
        for k in range(spike_idx[j]+1,spike_idx[j+1]):
            con_clean += clean_seq[k][:-1]
        clean_dict[re1.search(clean_seq[spike_idx[j]]).group()]=con_clean
    
    
    for i in range(0,1): # for many files, num_sam can be array(or list)
        path = os.path.join(root, 'ERR%d.sam' %num_sam)
    
    noisy_data = open(path,'r')
    noisy_seq = noisy_data.readlines()[94:2000]
    noisy_data.close()
    
    ## make header array ##
    
    err_count=0
    HD1 = []
    HD2 = []
    HD3 = []
    HD4 = []
    HD5 = []
    HD6 = []
    print "Pre-processing starts..."
    for j in range(len(noisy_seq)):
        if j%1000000==0:
                print "Pre-processing is %0.1f%% done.."%(j/float(len(noisy_seq))*100.)
        tmp = noisy_seq[j][19:180]
        head1 = re1.search(tmp) # ref. spike name
        head2 = re2.findall(tmp) # match pos. of clean_seq
        head3 = re3.search(tmp) # 30S70M
        head4 = re4.search(tmp) # noisy_seq. length=100
        
        cigar_str = head3.group()
        noisy = head4.group()
        
        if 'D' in cigar_str or 'I' in cigar_str or '*' in cigar_str: # Deletion & Insertion error pop up
            err_count += 1
        elif head1==None: # Error seq pop up
            err_count += 1
        elif 'N' in noisy: # Error seq pop up
            err_count += 1
        else:
            r_name = head1.group()
            tot_seq = clean_dict[r_name]
            tmp_seq = tot_seq[int(head2[2])-1:int(head2[2])-1+100] # len = 100
            m_idx = cigar_str.index('M')
            
            if m_idx == 1: # *M
                match_idx = int(cigar_str[m_idx-1])
                noisy = noisy[:match_idx]
                cut_seq = tmp_seq[:match_idx]
            elif m_idx == 2: # **M
                match_idx = int(cigar_str[m_idx-2]+cigar_str[m_idx-1])
                noisy = noisy[:match_idx]
                cut_seq = tmp_seq[:match_idx]
            elif m_idx == 3: 
                if cigar_str.count('S'): # *S*M
                    s_idx = cigar_str.index('S')
                    match_idx = int(cigar_str[s_idx+1])
                    ss_idx = int(cigar_str[s_idx-1])
                    noisy = noisy[ss_idx:ss_idx+match_idx]
                    cut_seq = tmp_seq[ss_idx:ss_idx+match_idx]
                else:
                    match_idx = 100 # ***M
                    noisy = noisy[:match_idx]
                    cut_seq = tmp_seq[:match_idx]
            elif m_idx == 4: 
                if cigar_str.count('S'): 
                    s_idx = cigar_str.index('S')
                    if s_idx == 1: # *S**M*S
                        match_idx = int(cigar_str[s_idx+1]+cigar_str[m_idx-1])
                        ss_idx = int(cigar_str[s_idx-1])
                        noisy = noisy[ss_idx:ss_idx+match_idx]
                        cut_seq = tmp_seq[ss_idx:ss_idx+match_idx]
                    elif s_idx == 2: # **S*M
                        match_idx = int(cigar_str[s_idx+1])
                        ss_idx = int(cigar_str[s_idx-2]+cigar_str[s_idx-1])
                        noisy = noisy[ss_idx:ss_idx+match_idx]
                        cut_seq = tmp_seq[ss_idx:ss_idx+match_idx]
            elif m_idx == 5: # **S**M
                if cigar_str.count('S'):
                    s_idx = cigar_str.index('S')
                    match_idx = int(cigar_str[s_idx+1]+cigar_str[s_idx+2])
                    ss_idx = int(cigar_str[s_idx-2]+cigar_str[s_idx-1])
                    noisy = noisy[ss_idx:ss_idx+match_idx]
                    cut_seq = tmp_seq[ss_idx:ss_idx+match_idx]
            else:
                print "There is another case!!"
                break
                
            HD1.append(r_name)
            HD2.append(head2[2])
            HD3.append(cigar_str)
            HD4.append(cut_seq)
            HD5.append(noisy)
            HD6.append(int(len(cut_seq)))
            
    print "Reshaping..."
    reshaped1 = np.reshape(HD1, (len(noisy_seq)-err_count, 1))
    reshaped2 = np.reshape(HD2, (len(noisy_seq)-err_count, 1))
    reshaped3 = np.reshape(HD3, (len(noisy_seq)-err_count, 1))
    reshaped4 = np.reshape(HD4, (len(noisy_seq)-err_count, 1))
    reshaped5 = np.reshape(HD5, (len(noisy_seq)-err_count, 1))
    reshaped6 = np.reshape(HD6, (len(noisy_seq)-err_count, 1))
    
    new_arr = np.hstack((reshaped1,reshaped2,reshaped3,reshaped4,reshaped5,reshaped6))
    new_arr = sorted(new_arr, key=itemgetter(0))
    new_arr = np.reshape(new_arr, (len(noisy_seq)-err_count, num_head))
    print "Pre-processing is completed!!"
    return new_arr # list of strs

def make_PI(new_arr):
    PI = zeros((4,4))
    dict_A = {'A':0, 'T':0, 'G':0, 'C':0}
    dict_T = {'A':0, 'T':0, 'G':0, 'C':0}
    dict_G = {'A':0, 'T':0, 'G':0, 'C':0}
    dict_C = {'A':0, 'T':0, 'G':0, 'C':0}
    tot_dict = [dict_A, dict_T, dict_G, dict_C]
    print "Making PI starts..."
    for i in range(len(new_arr)):
        if i%1000000==0:
            print "Making PI is %0.1f%% done.."%(i/float(len(new_arr))*100.) 
        for j in range(len(new_arr[i][3])):
            if new_arr[i][3][j] == 'A':
                if new_arr[i][4][j] == 'A':
                    dict_A[new_arr[i][4][j]] += 1
                elif new_arr[i][4][j] == 'T':
                    dict_A[new_arr[i][4][j]] += 1
                elif new_arr[i][4][j] == 'G':
                    dict_A[new_arr[i][4][j]] += 1
                elif new_arr[i][4][j] == 'C':
                    dict_A[new_arr[i][4][j]] += 1
                else:
                    print "Error 1"
                    break
            
            elif new_arr[i][3][j] == 'T':
                if new_arr[i][4][j] == 'A':
                    dict_T[new_arr[i][4][j]] += 1
                elif new_arr[i][4][j] == 'T':
                    dict_T[new_arr[i][4][j]] += 1
                elif new_arr[i][4][j] == 'G':
                    dict_T[new_arr[i][4][j]] += 1
                elif new_arr[i][4][j] == 'C':
                    dict_T[new_arr[i][4][j]] += 1
                else:
                    print "Error 2"
                    break
            
            elif new_arr[i][3][j] == 'G':
                if new_arr[i][4][j] == 'A':
                    dict_G[new_arr[i][4][j]] += 1
                elif new_arr[i][4][j] == 'T':
                    dict_G[new_arr[i][4][j]] += 1
                elif new_arr[i][4][j] == 'G':
                    dict_G[new_arr[i][4][j]] += 1
                elif new_arr[i][4][j] == 'C':
                    dict_G[new_arr[i][4][j]] += 1
                else:
                    print "Error 3"
                    break
            
            elif new_arr[i][3][j] == 'C':
                if new_arr[i][4][j] == 'A':
                    dict_C[new_arr[i][4][j]] += 1
                elif new_arr[i][4][j] == 'T':
                    dict_C[new_arr[i][4][j]] += 1
                elif new_arr[i][4][j] == 'G':
                    dict_C[new_arr[i][4][j]] += 1
                elif new_arr[i][4][j] == 'C':
                    dict_C[new_arr[i][4][j]] += 1
                else:
                    print "Error 4"
                    break
            else:
                print "Error 5"
                break
    
    for p in range(PI.shape[0]):
        PI[p,0] = tot_dict[p]['A']
        PI[p,1] = tot_dict[p]['T']
        PI[p,2] = tot_dict[p]['G']
        PI[p,3] = tot_dict[p]['C']
    for q in range(PI.shape[0]):
        PI[q] = PI[q]/float(sum(PI[q]))
    print "Making PI is completed!!"
    return PI

In [20]:
new = noisy_head_info('./',1211177, 92)
PI = make_PI(new)

Pre-processing starts...
Pre-processing is 0.0% done..
Reshaping...
Pre-processing is completed!!
Making PI starts...
Making PI is 0.0% done..
Making PI is completed!!


In [21]:
print new[:,5]
tmp = []
for i in range(int(new.shape[0]/5.)):
    tmp.append(int(new[i][5]))
print sum(tmp)

['52' '52' '34' ..., '98' '100' '100']
35930


In [16]:
with open("fold1/Illumina_LinErr_100_fold1.fasta", "r") as f:
    data = f.readlines()

In [31]:
print new[250071]
print new[250071][4]
print data[250071]
print len(data)
print new.shape[0]

['ERCC-00002' '837' '79M21S'
 'GACTTGTAAGGACCGCCGCTTTCGCTCGGGTCTGCGGGTTATAGCTTTTCAGTCTCGACGGGCTAGCACACATCTGGTT'
 'GACTCGTAAGGACCGCCGCTTTCGCTCGGGTCTGCGGGTTATAGCTTTTCAGTCTCGACGGGCTAGCACACATCTGGTT'
 '79']
GACTCGTAAGGACCGCCGCTTTCGCTCGGGTCTGCGGGTTATAGCTTTTCAGTCTCGACGGGCTAGCACACATCTGGTT
TACTGCCAAGCGAGCGTGGTGTGGTGCCCCCGACCTACAACTTGCGCCAATTATCGAGCTGGTAGACGACCAGCGCTGACGAGCTGGCGCAATGACGACC

491186
5954254


In [28]:
print int(len(data)/2)*150*2*1*4*4,"Bytes"
print int(len(data)/2)*150*2*1*4*4/float(1024),"MBytes"
print int(len(data)/2)*150*2*1*4*4/float(1024)/float(1024),"Gbytes"

1178846400 Bytes
1151217.1875 MBytes
1124.23553467 Gbytes


In [33]:
print 2977126/float(new.shape[0])


0.499999832053


In [36]:
print 24487/float(new.shape[0])

0.499969373379


In [39]:
print new[1]
print new[1][4]
print len(new[1][4])

['ERCC-00002' '872' '52M48S'
 'GGGTTATAGCTTTTCAGTCTCGACGGGCTAGCACACATCTGGTTGACTAGGC'
 'GGGTTATAGCTTTTCAGTCGCGACGGGCTAGCACACATCTGGTTTACTAGGC' '52']
GGGTTATAGCTTTTCAGTCGCGACGGGCTAGCACACATCTGGTTTACTAGGC
52
